In [1]:
%pip install tensorflow setuptools pillow scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
base_dir = 'Dataset/organs/'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

print(train_dir, val_dir, test_dir)

Dataset/organs/train Dataset/organs/val Dataset/organs/test


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2098 images belonging to 4 classes.
Found 263 images belonging to 4 classes.
Found 263 images belonging to 4 classes.


In [6]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
num_classes = train_generator.num_classes
output_layer = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy*100:.2f}%')

In [10]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

Epoch 1/15


d:\Codes\remote\projects\greeneye-ml\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 69s 904ms/step - accuracy: 0.5576 - loss: 1.1874 - val_accuracy: 0.7719 - val_loss: 0.5929
Epoch 2/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 60s 868ms/step - accuracy: 0.7952 - loss: 0.5099 - val_accuracy: 0.7795 - val_loss: 0.6032
Epoch 3/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 58s 841ms/step - accuracy: 0.8404 - loss: 0.3994 - val_accuracy: 0.7681 - val_loss: 0.6843
Epoch 4/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 59s 860ms/step - accuracy: 0.8487 - loss: 0.3610 - val_accuracy: 0.7757 - val_loss: 0.7048
Epoch 5/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 64s 918ms/step - accuracy: 0.8281 - loss: 0.4047 - val_accuracy: 0.7871 - val_loss: 0.5680
Epoch 6/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 62s 892ms/step - accuracy: 0.8743 - loss: 0.3134 - val_accuracy: 0.7909 - val_loss: 0.5630
Epoch 7/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 59s 853ms/step - accuracy: 0.8795 - loss: 0.3253 - val_accuracy: 0.7909 - val_loss: 0.5435
Epoch 8/15
66/66 ━━━━━━━━━━━━━━━━━━━━ 62s 891ms/step - accuracy: 0.8693 - loss: 0.3172 - val_accuracy: 0.802

In [11]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy * 100:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 518ms/step - accuracy: 0.7780 - loss: 0.5712
Test accuracy: 80.99%


In [3]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model("models/plant_organs_classifier.keras")

# Define image size
img_height, img_width = 224, 224

# Load and preprocess the image
img_path = "test/pohon-nangka.jpg"
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Predict the class of the image
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

# # Assuming you have a class indices dictionary from your train_generator
class_labels = list(train_generator.class_indices.keys())

print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: 2


In [13]:
model.save('plant_organs_classifier.keras')